In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import GRU, Linear, ReLU, Tanh, Dropout
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import ndcg_score
import emoji

# XAI ライブラリのインポート
import captum.attr as captum_attr
import shap
import lime
import lime.lime_tabular

# --- 1. 定数定義 ---
PREPROCESSED_FILE = 'posts_2017.csv'
HASHTAGS_FILE = 'hashtags_2017.csv'
MENTIONS_FILE = 'output_mentions_all_parallel.csv'
INFLUENCERS_FILE = 'influencers.txt'
MODEL_SAVE_PATH = f'influencer_rank_model_{time.strftime("%Y%m%d")}_rich_features_2017_3rd.pth'
FEATURE_DIM = 0 # グローバル変数として初期化

# (ここまでの関数は変更なし: prepare_graph_data, GCNEncoder, etc.)
# ... (既存のコードをここに貼り付け) ...
# --- 1. 定数定義 ---
PREPROCESSED_FILE = 'posts_2017.csv'
HASHTAGS_FILE = 'hashtags_2017.csv'
MENTIONS_FILE = 'output_mentions_all_parallel.csv'
INFLUENCERS_FILE = 'influencers.txt'
MODEL_SAVE_PATH = f'influencer_rank_model_{time.strftime("%Y%m%d")}_rich_features_2017_3rd.pth'

# --- 2. データ準備関数 (修正あり) ---
def prepare_graph_data(end_date, num_months=12, metric_numerator='likes', metric_denominator='posts'):
    """
    指定された終了日までのNヶ月間のグラフデータセットを構築する。
    指定された期間に活動のあったインフルエンサーのみを対象とする。
    """
    print(f"\nBuilding graph sequence for {num_months} months ending on {end_date.strftime('%Y-%m')}...")
    print(f"Using Engagement Metric: {metric_numerator} / {metric_denominator}")
    
    # --- データ読み込み ---
    df_posts = pd.read_csv(PREPROCESSED_FILE, parse_dates=['datetime'], low_memory=False)
    if 'comments' not in df_posts.columns: df_posts['comments'] = 0
    
    df_hashtags = pd.read_csv(HASHTAGS_FILE, header=0, low_memory=False)
    # ✅ この行を追加してカラム名を修正する
    df_hashtags.rename(columns={'source': 'username', 'target': 'hashtag'}, inplace=True) 

    df_mentions = pd.read_csv(MENTIONS_FILE, header=0, low_memory=False)
    # メンションの処理は元々コードに存在
    df_mentions.rename(columns={'source': 'username', 'target': 'mention'}, inplace=True)
    with open(INFLUENCERS_FILE, 'r', encoding='utf-8') as f: lines = f.readlines()
    lines = [line for line in lines if '===' not in line]
    from io import StringIO
    df_influencers_master = pd.read_csv(StringIO("".join(lines)), sep='\t', dtype=str)
    df_influencers_master.rename(columns={'#Followers': 'followers', '#Followees': 'followees', '#Posts': 'posts', 'Username': 'username', 'Category': 'category'}, inplace=True)

    df_hashtags['datetime'] = pd.to_datetime(df_hashtags['timestamp'], unit='s', errors='coerce').dropna()
    df_mentions['datetime'] = pd.to_datetime(df_mentions['timestamp'], unit='s', errors='coerce').dropna()
    df_posts['month'] = df_posts['datetime'].dt.to_period('M').dt.start_time

    # ✅✅✅ --- 変更点：活動のあったインフルエンサーのみに絞り込む --- ✅✅✅
    # 1. posts_2017.csv から,2017年に一度でも投稿したユーザーのリストを作成
    active_influencers_set = set(df_posts['username'].unique())
    print(f"Found {len(active_influencers_set):,} active influencers in {PREPROCESSED_FILE}.")

    # 2. influencers.txt のリストを,活動のあったユーザーのみにフィルタリング
    df_influencers = df_influencers_master[df_influencers_master['username'].isin(active_influencers_set)].copy()
    # --- 修正ここまで ---

    # --- ノードの準備 (フィルタリングされたリストを使用) ---
    influencer_set = set(df_influencers['username'].astype(str)) # これは active_influencers_set と同じはず
    all_hashtags = set(df_hashtags['hashtag'].astype(str))
    all_mentions = set(df_mentions['mention'].astype(str))
    all_nodes = sorted(list(influencer_set | all_hashtags | all_mentions))
    node_to_idx = {node: i for i, node in enumerate(all_nodes)}
    influencer_indices = [node_to_idx[inf] for inf in influencer_set if inf in node_to_idx]

    # --- 特徴量エンジニアリング ---
    node_df = pd.DataFrame({'username': all_nodes})
    profile_features = pd.merge(node_df, df_influencers[['username', 'followers', 'followees', 'posts', 'category']], on='username', how='left')
    for col in ['followers', 'followees', 'posts']:
        profile_features[col] = pd.to_numeric(profile_features[col], errors='coerce').fillna(0)
    category_dummies = pd.get_dummies(profile_features['category'], prefix='cat', dummy_na=True)
    profile_features = pd.concat([profile_features, category_dummies], axis=1).drop(columns=['category'])
    node_df['type'] = 'other_user'
    node_df.loc[node_df['username'].isin(influencer_set), 'type'] = 'influencer'
    node_df.loc[node_df['username'].isin(all_hashtags), 'type'] = 'hashtag'
    node_type_dummies = pd.get_dummies(node_df['type'], prefix='type')
    static_features = pd.concat([profile_features, node_type_dummies], axis=1)

    df_posts['emoji_count'] = df_posts['caption'].astype(str).apply(emoji.emoji_count)
    df_posts.sort_values(by=['username', 'datetime'], inplace=True)
    df_posts['post_interval_sec'] = df_posts.groupby('username')['datetime'].diff().dt.total_seconds()
    post_categories = [f'post_cat_{i}' for i in range(10)]
    df_posts['post_category'] = np.random.choice(post_categories, size=len(df_posts))
    df_posts['is_ad'] = np.random.choice([0, 1], size=len(df_posts), p=[0.9, 0.1])
    
    dynamic_agg = df_posts.groupby(['username', 'month']).agg(
        monthly_post_count=('datetime', 'size'), avg_caption_length=('caption', lambda x: x.astype(str).str.len().mean()),
        avg_tag_count=('tag_count', 'mean'), avg_sentiment=('sentiment', 'mean'),
        avg_emoji_count=('emoji_count', 'mean'), avg_post_interval=('post_interval_sec', 'mean'),
        ad_rate=('is_ad', 'mean')).reset_index()
    post_category_rate = df_posts.groupby(['username', 'month'])['post_category'].value_counts(normalize=True).unstack(fill_value=0)
    post_category_rate.columns = [f'rate_{col}' for col in post_category_rate.columns]
    dynamic_features = pd.merge(dynamic_agg, post_category_rate, on=['username', 'month'], how='left')
    
    monthly_graphs = []
    start_date = end_date - pd.DateOffset(months=num_months-1)
    
    feature_columns = list(static_features.drop('username', axis=1).columns) + list(dynamic_features.drop(['username', 'month'], axis=1).columns) + ['feedback_rate']
    
    global FEATURE_DIM
    FEATURE_DIM = len(feature_columns)
    print(f"Total feature dimension: {FEATURE_DIM}")

    for snapshot_date in tqdm(pd.date_range(start_date, end_date, freq='ME'), desc="Building monthly graphs"):
        snapshot_month = snapshot_date.to_period('M').start_time
        current_hashtags = df_hashtags[df_hashtags['datetime'] <= snapshot_date]
        current_mentions = df_mentions[df_mentions['datetime'] <= snapshot_date]
        edges_ht = [(node_to_idx[str(u)], node_to_idx[str(h)]) for u, h in zip(current_hashtags['username'], current_hashtags['hashtag']) if str(u) in node_to_idx and str(h) in node_to_idx]
        edges_mt = [(node_to_idx[str(u)], node_to_idx[str(m)]) for u, m in zip(current_mentions['username'], current_mentions['mention']) if str(u) in node_to_idx and str(m) in node_to_idx]
        if not edges_ht and not edges_mt: continue
        edge_index = torch.tensor(list(set(edges_ht + edges_mt)), dtype=torch.long).t().contiguous()
        
        current_dynamic = dynamic_features[dynamic_features['month'] == snapshot_month]
        snapshot_features = pd.merge(static_features, current_dynamic, on='username', how='left')
        snapshot_features['feedback_rate'] = 0.0
        snapshot_features = snapshot_features[feature_columns].fillna(0)
        
        x = torch.tensor(snapshot_features.astype(float).values, dtype=torch.float)        
        monthly_posts_period = df_posts[df_posts['datetime'].dt.to_period('M') == snapshot_date.to_period('M')]
        monthly_agg = monthly_posts_period.groupby('username').agg(
            total_likes=('likes', 'sum'), total_comments=('comments', 'sum'), post_count=('datetime', 'size')).reset_index()
        
        if metric_numerator == 'likes_and_comments': monthly_agg['numerator'] = monthly_agg['total_likes'] + monthly_agg['total_comments']
        else: monthly_agg['numerator'] = monthly_agg['total_likes']
            
        if metric_denominator == 'followers':
            monthly_agg['avg_engagement_per_post'] = (monthly_agg['numerator'] / monthly_agg['post_count']).where(monthly_agg['post_count'] > 0, 0)
            merged_data = pd.merge(monthly_agg, static_features[['username', 'followers']], on='username', how='left')
            merged_data['engagement'] = (merged_data['avg_engagement_per_post'] / merged_data['followers']).where(merged_data['followers'] > 0, 0)
        else:
            merged_data = monthly_agg
            merged_data['engagement'] = (merged_data['numerator'] / merged_data['post_count']).where(merged_data['post_count'] > 0, 0)
        
        engagement_data = pd.merge(pd.DataFrame({'username': all_nodes}), merged_data[['username', 'engagement']], on='username', how='left').fillna(0)
        y = torch.tensor(engagement_data['engagement'].values, dtype=torch.float).view(-1, 1)
        
        graph_data = Data(x=x, edge_index=edge_index, y=y)
        monthly_graphs.append(graph_data)
        
    return monthly_graphs, influencer_indices, node_to_idx


# --- モデル定義とその他の関数 (変更なし) ---
# (GCNEncoder, AttentiveRNN, InfluencerRankModel, BatchedListwiseRankingLoss, display_relevance_distribution, etc.)
class GCNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers=2):
        super(GCNEncoder, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList([GCNConv(in_channels, hidden_channels)] + [GCNConv(hidden_channels, hidden_channels) for _ in range(num_layers - 1)])
    def forward(self, x, edge_index):
        layer_outputs = []
        for conv in self.convs:
            x = conv(x, edge_index).relu()
            layer_outputs.append(x)
        return torch.cat(layer_outputs, dim=1)

class AttentiveRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AttentiveRNN, self).__init__()
        self.rnn = GRU(input_dim, hidden_dim, batch_first=True)
        self.attention_layer = Linear(hidden_dim, 1)
    def forward(self, sequence_of_embeddings):
        rnn_out, _ = self.rnn(sequence_of_embeddings)
        attention_scores = self.attention_layer(rnn_out).tanh()
        attention_weights = torch.softmax(attention_scores, dim=1)
        return torch.sum(rnn_out * attention_weights, dim=1)

class InfluencerRankModel(nn.Module):
    def __init__(self, feature_dim, gcn_dim, rnn_dim, num_gcn_layers=2, dropout_prob=0.5):
        super(InfluencerRankModel, self).__init__()
        self.gcn_encoder = GCNEncoder(feature_dim, gcn_dim, num_gcn_layers)
        self.attentive_rnn = AttentiveRNN(gcn_dim * num_gcn_layers, rnn_dim)
        self.predictor = nn.Sequential(Linear(rnn_dim, 16), ReLU(), Dropout(dropout_prob), Linear(16, 1))
    # ✅ forwardメソッドを追加してモデルの処理フローを明確化
    def forward(self, graph_sequence, target_indices):
        sequence_embeddings = torch.stack([self.gcn_encoder(g.x, g.edge_index) for g in graph_sequence])
        target_embeddings = sequence_embeddings[:, target_indices].permute(1, 0, 2)
        final_representation = self.attentive_rnn(target_embeddings)
        predicted_scores = self.predictor(final_representation)
        return predicted_scores

class BatchedListwiseRankingLoss(nn.Module):
    def __init__(self):
        super(BatchedListwiseRankingLoss, self).__init__()
    def forward(self, pred_scores, true_scores):
        pred_probs = F.softmax(pred_scores, dim=1)
        true_probs = F.softmax(true_scores, dim=1)
        return -torch.sum(true_probs * torch.log(pred_probs + 1e-9), dim=1).mean()

def display_relevance_distribution(scores, title):
    scores_series = pd.Series(scores)
    relevance_series = scores_series.apply(assign_relevance_levels)
    counts = relevance_series.value_counts().sort_index()
    percentages = relevance_series.value_counts(normalize=True).sort_index() * 100
    dist_df = pd.DataFrame({'Relevance': counts.index, 'Count': counts.values, 'Percentage': percentages.values}).set_index('Relevance')
    dist_df = dist_df.reindex(range(6), fill_value=0)
    dist_df['Percentage'] = dist_df['Percentage'].map('{:.2f}%'.format)
    print(f"\n--- {title} ---")
    print(dist_df)

def assign_relevance_levels(engagement_rate):
    if engagement_rate >= 0.10: return 5
    if engagement_rate >= 0.07: return 4
    if engagement_rate >= 0.05: return 3
    if engagement_rate >= 0.03: return 2
    if engagement_rate >= 0.01: return 1
    return 0

def calculate_rbp(true_scores_in_predicted_order, p=0.95):
    rbp_score = 0
    max_score = true_scores_in_predicted_order.max()
    if max_score == 0: return 0.0
    normalized_scores = true_scores_in_predicted_order / max_score
    for i, relevance in enumerate(normalized_scores):
        rbp_score += (p ** i) * relevance
    return (1 - p) * rbp_score


def train_and_save_model():
    # False で 0.001
    END_TO_END_TRAINING = False
    GCN_DIM = 128
    NUM_GCN_LAYERS = 2
    RNN_DIM = 64
    LEARNING_RATE = 0.001
    DROPOUT_PROB = 0.5
    NUM_EPOCHS = 200
    LISTS_PER_BATCH = 1024
    LIST_SIZE = 10
    BATCH_SIZE = LISTS_PER_BATCH * LIST_SIZE
    METRIC_NUMERATOR = 'likes_and_comments'
    METRIC_DENOMINATOR = 'followers'

    print(f"--- Starting Training ---")
    start_time = time.time()
    
    latest_date = pd.to_datetime('2017-12-31')
    # ✅ prepare_graph_dataがFEATURE_DIMを設定する
    monthly_graphs, influencer_indices, _ = prepare_graph_data(end_date=latest_date, num_months=12, metric_numerator=METRIC_NUMERATOR, metric_denominator=METRIC_DENOMINATOR)
    if not monthly_graphs:
        print("No graph data was created. Exiting.")
        return

    # ✅ feature_dimをグローバル変数から取得
    model = InfluencerRankModel(feature_dim=FEATURE_DIM, gcn_dim=GCN_DIM, rnn_dim=RNN_DIM, num_gcn_layers=NUM_GCN_LAYERS, dropout_prob=DROPOUT_PROB)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion_listwise = BatchedListwiseRankingLoss()
    
    criterion_pointwise = nn.MSELoss() 
    alpha = 1 # 2つの損失のバランスを取るための重み（調整可能）
    
    true_scores = monthly_graphs[-1].y[influencer_indices]
    display_relevance_distribution(true_scores.squeeze().cpu().numpy(), "📊 Training Data Ground Truth Distribution")
    dataset = TensorDataset(torch.tensor(influencer_indices, dtype=torch.long), true_scores)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
    
    # (学習ループは変更なし)
    if not END_TO_END_TRAINING:
        print("\n--- Strategy: Two-Stage Learning (Fast) ---")
        model.gcn_encoder.eval()
        with torch.no_grad():
            sequence_embeddings = torch.stack([model.gcn_encoder(g.x, g.edge_index) for g in tqdm(monthly_graphs, desc="GCN Encoding")])
        model.attentive_rnn.train()
        model.predictor.train()
        for epoch in range(NUM_EPOCHS):
            total_loss = 0
            for batch_indices, batch_true_scores in tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
                optimizer.zero_grad()
                batch_sequence_embeddings = sequence_embeddings[:, batch_indices].permute(1, 0, 2)
                final_user_representation = model.attentive_rnn(batch_sequence_embeddings)
                predicted_scores = model.predictor(final_user_representation)
                predicted_scores_reshaped = predicted_scores.view(LISTS_PER_BATCH, LIST_SIZE)
                batch_true_scores_reshaped = batch_true_scores.view(LISTS_PER_BATCH, LIST_SIZE)
                loss_listwise = criterion_listwise(predicted_scores_reshaped, batch_true_scores_reshaped)
                loss_pointwise = criterion_pointwise(predicted_scores.squeeze(), batch_true_scores.squeeze())
                loss = loss_listwise + alpha * loss_pointwise
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Average Batch Loss: {total_loss / len(dataloader):.4f}")
    else:
        # --- 戦略2: エンドツーエンド学習 ---
        print("\n--- Strategy: End-to-End Learning (Slow, High-Memory) ---")
        model.train() # モデル全体を訓練モードに設定
        
        for epoch in range(NUM_EPOCHS):
            # GCN計算をエポックの開始時に実行
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}: Performing GCN forward pass for this epoch...")
            # 勾配計算を有効にしたままフォワードパスを実行
            sequence_embeddings = torch.stack([model.gcn_encoder(g.x, g.edge_index) for g in monthly_graphs])
            
            total_loss = 0
            for batch_indices, batch_true_scores in tqdm(dataloader, desc=f"Training Batches"):
                optimizer.zero_grad()
                
                # エポックごとに計算した特徴量を使用
                batch_sequence_embeddings = sequence_embeddings[:, batch_indices].permute(1, 0, 2)
                final_user_representation = model.attentive_rnn(batch_sequence_embeddings)
                predicted_scores = model.predictor(final_user_representation)

                # ✅ --- 変更点: エンドツーエンドでも2種類の損失を計算 ---
                # 1. リストワイズ損失
                predicted_scores_reshaped = predicted_scores.view(LISTS_PER_BATCH, LIST_SIZE)
                batch_true_scores_reshaped = batch_true_scores.view(LISTS_PER_BATCH, LIST_SIZE)
                loss_listwise = criterion_listwise(predicted_scores_reshaped, batch_true_scores_reshaped)
                
                # 2. ポイントワイズ損失
                loss_pointwise = criterion_pointwise(predicted_scores.squeeze(), batch_true_scores.squeeze())
                
                # 3. 2つの損失を合計
                loss = loss_listwise + alpha * loss_pointwise

                # 誤差がGCNまで逆伝播し,モデル全体の重みが更新される
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Average Batch Loss: {total_loss / len(dataloader):.4f}")
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    end_time = time.time()
    print("\n--- Training Complete ---")
    print(f"✅ Model saved to '{MODEL_SAVE_PATH}'")
    print(f"Total time: {end_time - start_time:.2f} seconds")


def run_inference():
    METRIC_NUMERATOR = 'likes_and_comments'
    METRIC_DENOMINATOR = 'followers'
    
    print("--- 📈 Starting Inference Process ---")
    start_time = time.time()
    params = {'GCN_DIM': 128, 'NUM_GCN_LAYERS': 2, 'RNN_DIM': 64, 'DROPOUT_PROB': 0.5}

    latest_date = pd.to_datetime('2017-12-31')
    # ✅ 推論時もFEATURE_DIMが設定される
    predict_graphs, predict_indices, node_to_idx = prepare_graph_data(
        end_date=latest_date, num_months=12, metric_numerator=METRIC_NUMERATOR, metric_denominator=METRIC_DENOMINATOR)
    
    # ✅ feature_dimをグローバル変数から取得
    model = InfluencerRankModel(feature_dim=FEATURE_DIM, gcn_dim=params['GCN_DIM'], rnn_dim=params['RNN_DIM'], num_gcn_layers=params['NUM_GCN_LAYERS'], dropout_prob=params['DROPOUT_PROB'])
    try:
        model.load_state_dict(torch.load(MODEL_SAVE_PATH))
        print(f"Successfully loaded model from '{MODEL_SAVE_PATH}'")
    except FileNotFoundError:
        print(f"Error: Model file not found at '{MODEL_SAVE_PATH}'. Please run training first.")
        return

    inference_input_graphs = predict_graphs[:-1]
    ground_truth_graph = predict_graphs[-1]

    model.eval()
    with torch.no_grad():
        # ✅ モデルのforwardメソッドを使って予測を簡潔に記述
        predicted_scores = model(inference_input_graphs, predict_indices)

    # (結果の集計と表示部分は変更なし)
    idx_to_node = {i: node for node, i in node_to_idx.items()}
    influencer_usernames = [idx_to_node[idx] for idx in predict_indices]
    true_scores = ground_truth_graph.y[predict_indices]
    
    df_results = pd.DataFrame({
        'Username': influencer_usernames,
        'Predicted_Score': predicted_scores.squeeze().cpu().numpy(),
        'True_Score': true_scores.squeeze().cpu().numpy()
    })
    
    mae = (df_results['Predicted_Score'] - df_results['True_Score']).abs().mean()
    mse = ((df_results['Predicted_Score'] - df_results['True_Score']) ** 2).mean()
    rmse = np.sqrt(mse)
    
    df_results['Relevance'] = df_results['True_Score'].apply(assign_relevance_levels)
    true_relevance = df_results['Relevance'].values.reshape(1, -1)
    predicted_scores_for_ndcg = df_results['Predicted_Score'].values.reshape(1, -1)
    
    ndcg_results = {}
    k_values = [1, 10, 50, 100, 200]
    for k in k_values:
        if k > len(df_results): continue
        ndcg_results[f'NDCG@{k}'] = ndcg_score(true_relevance, predicted_scores_for_ndcg, k=k)
    
    df_sorted_by_pred = df_results.sort_values(by='Predicted_Score', ascending=False)
    true_scores_in_pred_order = df_sorted_by_pred['True_Score'].values
    rbp_val = calculate_rbp(true_scores_in_pred_order, p=0.95)

    df_results['Predicted_Rank'] = df_results['Predicted_Score'].rank(ascending=False, method='first').astype(int)
    
    print("\n🏆 --- Top 20 Predicted Influencers (with True Scores) --- 🏆")
    print(df_results.sort_values(by='Predicted_Rank')[['Username', 'Predicted_Score', 'True_Score']].head(20).to_string(index=False))
    
    print("\n\n" + "="*50)
    print("📊 MODEL PERFORMANCE EVALUATION")
    print("="*50)
    
    display_relevance_distribution(df_results['True_Score'], "📈 Inference Data Ground Truth Distribution")
    display_relevance_distribution(df_results['Predicted_Score'], "🤖 Inference Data Predicted Distribution")

    print("\n🎯 --- A. Prediction Accuracy Metrics (値の正確さ) ---")
    print(f"   - **MAE (平均絶対誤差)**: {mae:.4f}")
    print(f"   - **RMSE (二乗平均平方根誤差)**: {rmse:.4f}")

    print("\n🏅 --- B. Ranking Quality Metrics (順序の正しさ) ---")
    print(f"   - **NDCG@K (正規化割引累積利得)**:")
    for k_str, score in ndcg_results.items():
        print(f"     - {k_str:<8}: {score:.4f}")

    print(f"\n   - **RBP (ランクバイアス適合率)**: {rbp_val:.4f}")
    
    end_time = time.time()
    print(f"\nTotal inference time: {end_time - start_time:.2f} seconds")
    
    # XAI分析のために結果を返す
    return model, predict_graphs, predict_indices, node_to_idx, df_results


# --- 乱数シード設定関数 ---
def set_seed(seed_value=42):
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value)
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True # CUDNNの非決定性を抑制
        torch.backends.cudnn.benchmark = False   # CUDNNの自動ベンチマークを無効化

# ==============================================================================
# 🧠 --- 5. XAI分析関数 (新規追加) ---
# ==============================================================================

class RnnExplainerModel(nn.Module):
    """XAIのためにRNNとPredictorをラップするモデル"""
    def __init__(self, attentive_rnn, predictor):
        super().__init__()
        self.attentive_rnn = attentive_rnn
        self.predictor = predictor

    def forward(self, rnn_input_sequence):
        # 入力は (batch, seq_len, features)
        final_representation = self.attentive_rnn(rnn_input_sequence)
        prediction = self.predictor(final_representation)
        return prediction

def run_xai_analysis(model, graph_sequence, influencer_indices, node_to_idx, df_results):
    """
    指定されたインフルエンサーについて,RNNへの貢献度を3つの手法で分析する
    """
    print("\n\n" + "="*50)
    print("🧠 XAI ANALYSIS: Explaining RNN Contributions")
    print("="*50)

    # --- 1. 分析対象の準備 ---
    # 最もスコアが高いと予測されたインフルエンサーを分析対象に選ぶ
    target_username = df_results.sort_values(by='Predicted_Score', ascending=False).iloc[0]['Username']
    target_idx_in_list = [i for i, idx in enumerate(influencer_indices) if node_to_idx.get(target_username) == idx][0]
    
    print(f"🎯 Explaining prediction for top-ranked influencer: @{target_username}\n")

    model.eval()
    with torch.no_grad():
        # GCNで全インフルエンサーの時間ごとの特徴量（埋め込み）を計算
        sequence_embeddings = torch.stack([model.gcn_encoder(g.x, g.edge_index) for g in graph_sequence])
        
        # 分析対象のインフルエンサーの埋め込み系列を抽出
        # (seq_len, features) -> (1, seq_len, features) の形状にする
        input_for_rnn = sequence_embeddings[:, influencer_indices[target_idx_in_list]].unsqueeze(0)

    # --- 2. 説明用のサブモデルを定義 ---
    explainer_model = RnnExplainerModel(model.attentive_rnn, model.predictor)

    # --- 3. 各XAI手法を適用 ---
    
    # --- 3.1 Captum (Integrated Gradients) ---
    print("\n--- 📊 1. Captum (Integrated Gradients) ---")
    ig = captum_attr.IntegratedGradients(explainer_model)
    attributions_ig, delta = ig.attribute(input_for_rnn, return_convergence_delta=True)
    # 特徴量次元で合計し,月ごとの貢献度を算出
    monthly_attr_ig = attributions_ig.squeeze(0).sum(dim=1).cpu().numpy()
    
    for i, attr in enumerate(monthly_attr_ig):
        print(f"  Month {i+1:2d}: Contribution = {attr: .4f}")

    # --- 3.2 SHAP (GradientExplainer) ---
    print("\n--- 📊 2. SHAP (GradientExplainer) ---")
    # 背景データとして,他のインフルエンサーの埋め込み系列をランダムに50人分使用
    background_indices = np.random.choice(len(influencer_indices), 50, replace=False)
    background_data = sequence_embeddings[:, influencer_indices].permute(1, 0, 2)[background_indices]
    
    explainer_shap = shap.GradientExplainer(explainer_model, background_data)
    shap_values = explainer_shap.shap_values(input_for_rnn)
    # 特徴量次元で合計し,月ごとの貢献度を算出
    monthly_attr_shap = shap_values.sum(axis=2).flatten()

    for i, attr in enumerate(monthly_attr_shap):
        print(f"  Month {i+1:2d}: Contribution = {attr: .4f}")

    # --- 3.3 LIME (TabularExplainer) ---
    print("\n--- 📊 3. LIME (TabularExplainer) ---")
    
    # LIME用の予測関数
    def lime_predictor(numpy_array):
        # LIMEは (num_samples, num_features) の2D配列を渡す
        # ここでは12ヶ月 * 特徴量次元 = 256*12 = 3072 のようなフラットな配列になる
        num_samples = numpy_array.shape[0]
        # (num_samples, seq_len * features) -> (num_samples, seq_len, features)
        tensor_input = torch.from_numpy(numpy_array).float().view(num_samples, input_for_rnn.shape[1], -1)
        with torch.no_grad():
            predictions = explainer_model(tensor_input)
        return predictions.cpu().numpy()

    # LIME Explainerの準備
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=background_data.reshape(background_data.shape[0], -1).cpu().numpy(),
        mode='regression',
        feature_names=[f'Month_{i+1}_Emb_{j}' for i in range(input_for_rnn.shape[1]) for j in range(input_for_rnn.shape[2])],
        verbose=False
    )

    # 説明を生成
    # LIMEの入力はフラットな1D配列にする必要がある
    lime_explanation = lime_explainer.explain_instance(
        data_row=input_for_rnn.squeeze(0).flatten().cpu().numpy(),
        predict_fn=lime_predictor,
        num_features=12 # 月ごとの重要度を見たいので,トップ12個の特徴を表示
    )

    # 結果を月ごとに集計
    lime_monthly_contributions = {i: 0.0 for i in range(1, 13)}
    for feature, weight in lime_explanation.as_list():
        # 'Month_X_...' から月番号を抽出
        try:
            month = int(feature.split('_')[1])
            lime_monthly_contributions[month] += weight
        except (IndexError, ValueError):
            continue
            
    for month, total_weight in sorted(lime_monthly_contributions.items()):
        print(f"  Month {month:2d}: Contribution = {total_weight: .4f}")


# --- mainブロックの修正案 ---
if __name__ == '__main__':
    # ✅ この行を追加してシードを固定
    set_seed(42) 
    
    # train_and_save_model()
    # run_inference()からXAIに必要な情報を引き継ぐように修正
    model, graphs, indices, node_map, results_df = run_inference()
    
    # XAI分析を実行
    if model and graphs:
        # 推論で使用したグラフシーケンスは11ヶ月分なので,それを使用
        run_xai_analysis(model, graphs[:-1], indices, node_map, results_df)

--- Starting Training ---

Building graph sequence for 12 months ending on 2017-12...
Using Engagement Metric: likes_and_comments / followers
Found 20,588 active influencers in posts_2017.csv.
Total feature dimension: 36


Building monthly graphs: 100%|██████████| 12/12 [02:04<00:00, 10.40s/it]



--- 📊 Training Data Ground Truth Distribution ---
           Count Percentage
Relevance                  
0           1807      8.95%
1           6450     31.95%
2           5073     25.13%
3           2892     14.32%
4           2252     11.15%
5           1716      8.50%

--- Strategy: Two-Stage Learning (Fast) ---


Epoch 1/200: 100%|██████████| 1/1 [00:00<00:00,  4.33it/s]


Epoch 1/200, Average Batch Loss: 2.3847


...

Epoch 199/200, Average Batch Loss: 2.3043


Epoch 200/200: 100%|██████████| 1/1 [00:00<00:00,  4.44it/s]


Epoch 200/200, Average Batch Loss: 2.3043

--- Training Complete ---
✅ Model saved to 'influencer_rank_model_20251029_rich_features_2017_3rd.pth'
Total time: 525.41 seconds
--- 📈 Starting Inference Process ---

Building graph sequence for 12 months ending on 2017-12...
Using Engagement Metric: likes_and_comments / followers
Found 20,588 active influencers in posts_2017.csv.
Total feature dimension: 36


Building monthly graphs: 100%|██████████| 12/12 [02:02<00:00, 10.20s/it]
/tmp/ipykernel_870630/1276952809.py:361: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_st

Successfully loaded model from 'influencer_rank_model_20251029_rich_features_2017_3rd.pth'

🏆 --- Top 20 Predicted Influencers (with True Scores) --- 🏆
                 Username  Predicted_Score  True_Score
             macherstudio         0.079182    0.100776
              mikalakitty         0.070222    0.056238
         mamaliciousmaria         0.069036    0.008338
                  kyxy965         0.069002    0.018039
        somethingsotruesd         0.068842    0.019553
                kgmilller         0.067610    0.146686
           sophtheshopper         0.067096    0.057175
                   ahgfun         0.067060    0.062917
         homeoftheharveys         0.063709    0.085306
                   a4n2a0         0.063262    0.012711
       theglutenfreesloth         0.062813    0.061173
     hellomyraeofsunshine         0.062615    0.046588
        curvesandacarryon         0.062034    0.016944
               mygbgvlife         0.061987    0.024428
        theefunnyaquari